In [1]:
import os
from time import sleep

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as soup 
import holidays

### 0. Load Bike Sharing Data

**Origin of the Dataset**

The dataset used in this analysis is sourced from **Capital Bikeshare**, a bike-sharing service operating in the Washington, D.C. metropolitan area. The data includes information from the years 2010 to 2017 and provides valuable insights into bike-sharing usage patterns, trip durations, and other related factors.

**Description of Columns**

The dataset contains several columns that provide specific information about bike-sharing trips and related factors. Here's a description of some commonly found columns in the dataset:

- `Start Time`: The date and time when the trip started.
- `End Time`: The date and time when the trip ended.
- `Start Station ID`: The unique identifier of the station where the trip originated.
- `End Station ID`: The unique identifier of the station where the trip ended.
- `Start Station Name`: The name or label of the station where the trip originated.
- `End Station Name`: The name or label of the station where the trip ended.
- `Bike ID`: The unique identifier of the bike used for the trip.
- `Member Type`: Indicates whether the user was a registered member or a casual/non-member.

We will merge the datasets from the years 2010 to 2017 and compress the information into 4 columns of interest:
- `Date`: Date when the rental was initiated.
- `Hour`: Hour of the day (1-23) when the rental was initiated.
- `Member counts`: Number of rentals done by registered users.
- `Casual counts`: Number of rentals done by non-registered users.
- `Total`: Sum of member and casual counts.

In [3]:
data_path = os.path.join(os.pardir,'data', 'raw', 'bikesharing')
data_files = [file for file in os.listdir(data_path) if file.endswith('.csv')]

In [4]:
# Initialize empty dataframe that will store bike sharing data
bdf = pd.DataFrame(columns=['date', 'hour', 'member', 'casual', 'total'])

for file in data_files:
    
    # Print file being imported
    print(file)

    # Read file and preprocess columns
    df = pd.read_csv(os.path.join(data_path, file))
    df['Start date'] = pd.to_datetime(df['Start date'], errors='coerce')    
    df['Member type'] = df['Member type'].apply(lambda x: 'casual' if x == 'Unknown' else x.lower())
    
    # Group by date, hour, and member type to have hourly counts by member type
    grouped = df.groupby([df['Start date'].dt.date.rename('date'), df['Start date'].dt.hour.rename('hour'), df['Member type'].rename('member_type')]).size().reset_index(name='count')
    # Pivot column to get count by member type in the same row for the same date, hour
    pivoted = pd.pivot_table(grouped, values='count', index=['date', 'hour'], columns='member_type', fill_value=0).reset_index()
    # Compute total as the sum of counts for each member type 
    pivoted['total'] = pivoted['casual'] + pivoted['member']
    # Concat with previously imported bike sharing data
    bdf = pd.concat((bdf, pivoted)).sort_values(['date', 'hour'])

    bdf.date = pd.to_datetime(bdf.date, errors='coerce')
    # Save file for future use
    bdf.to_csv(os.path.join(os.pardir,'data', 'processed', 'bikesharing.csv'), index=False)

2016Q2-capitalbikeshare-tripdata.csv
2013Q2-capitalbikeshare-tripdata.csv
2015Q4-capitalbikeshare-tripdata.csv
2012Q1-capitalbikeshare-tripdata.csv
2017Q1-capitalbikeshare-tripdata.csv
2015Q3-capitalbikeshare-tripdata.csv
2013Q3-capitalbikeshare-tripdata.csv
2014Q1-capitalbikeshare-tripdata.csv
2016Q3-capitalbikeshare-tripdata.csv
2015Q2-capitalbikeshare-tripdata.csv
2013Q4-capitalbikeshare-tripdata.csv
2016Q4-capitalbikeshare-tripdata.csv
2017Q3-capitalbikeshare-tripdata.csv
2015Q1-capitalbikeshare-tripdata.csv
2010-capitalbikeshare-tripdata.csv
2012Q3-capitalbikeshare-tripdata.csv
2017Q4-capitalbikeshare-tripdata.csv
2012Q4-capitalbikeshare-tripdata.csv
2014Q2-capitalbikeshare-tripdata.csv
2014Q4-capitalbikeshare-tripdata.csv
2012Q2-capitalbikeshare-tripdata.csv
2011-capitalbikeshare-tripdata.csv
2017Q2-capitalbikeshare-tripdata.csv
2014Q3-capitalbikeshare-tripdata.csv
2016Q1-capitalbikeshare-tripdata.csv
2013Q1-capitalbikeshare-tripdata.csv


### 0.1 Load weather data

Here we will gather weather data that can help us predict the bikesharing demand using web scrapping methods to the website freemeteo. We will collect hourly weather data, including:
- `Temperature`
- `Relative temperature`
- `Windspeed`
- `Humidity`
- `Weather`:
  1. Clear, Few clouds, Partly cloudy, Partly cloudy
  2. Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
  3. Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
  4. Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
- `Details`: description of weather.

In [5]:

def get_weather(date):
    """Function to webscrape one day from freemeteo.bg at Washington DC station.
    Input: 'year-month-day' -- 2019-01-01
    Output: pandas dataframe
    
    """
    
    url = F"https://freemeteo.bg/weather/washington/history/daily-history/?gid=4140963&station=19064&date={date}&language=english&country=us-united-states"
    res =  requests.get(url)
    soup1 = soup(res.text, "lxml")

    columns = ['datetime', 'temp','atemp','windir','humidity','weather', 'details']
    df=pd.DataFrame(columns=columns)

    for row in soup1.findAll("tr"):

        cells = row.findAll("td")
        row = []
        if len(cells) ==10:

            time = cells[0].find(text=True)
            temp = cells[1].find(text=True)
            atemp = cells[2].find(text=True)
            windir = cells[3].text
            humidity = cells[5].find(text=True)
            weather = cells[8].span.attrs['data-icon']
            datetime = f'{date}T{time}'
            details = cells[9].find('span', "details").text

            row.append([datetime,temp,atemp,windir,humidity,weather, details])
            row = pd.DataFrame(row, columns=columns)
            row.datetime = pd.to_datetime(row.datetime, errors='coerce')
            row['date'] = row.datetime.dt.date
            row['hour'] = row.datetime.dt.hour

            df = df.append(row, ignore_index=True)
    
    df.temp = df.temp.str.replace('°C', '')
    df.atemp = df.atemp.str.replace('°C', '')
    df.humidity = df.humidity.str.replace('%', '')
    pattern = r'\b(\d+)\s+Km/h'
    df['windspeed'] = df.windir.str.extract(pattern, expand=False)
    df.windspeed = df.apply(lambda x: 0 if (pd.isnull(x.windspeed) and x.windir == 'Calm') else x.windspeed, axis=1)
    
    df.drop_duplicates(subset=['date', 'hour'], inplace=True)
    
    return df
     

In [6]:
wdf = pd.DataFrame(columns=['datetime', 'temp','atemp','windir','humidity','weather', 'details'])

# Obtain weather data for each date in bike sharing dataframe
for date in bdf.date.dt.strftime('%Y-%m-%d').unique():
    wdf = pd.concat((wdf, get_weather(date=date)))

wdf.date = pd.to_datetime(wdf.date)

# Group weather values 
# Identified icons > 4:
# 26 = snow
# 10 = rays
# 94 = fog
wdf.weather = wdf.weather.apply(lambda x: 4 if int(x) > 4 else (x))

# Save file for future use
wdf.to_csv(os.path.join(os.pardir,'data', 'processed', 'weather.csv'), index=False)

### 0.3 Load public holidays data

In [7]:
# Create a list of public holidays for Washington, D.C.
dc_holidays = holidays.US(state='DC')

# Generate a date range from 2009 to the current year
date_range = pd.date_range(start=bdf.date.min().strftime('%Y-%m-%d'), end=bdf.date.max().strftime('%Y-%m-%d'))

# Create a DataFrame with 'date' column
hdf = pd.DataFrame({'date': date_range})

# Check if each date is a public holiday or not
hdf['public_holiday'] = hdf['date'].apply(lambda x: 1 if x in dc_holidays else 0)

# Save for future use
hdf.to_csv(os.path.join(os.pardir,'data', 'processed', 'holidays.csv'), index=False)

### 0.4 Merge datasets

In [8]:
# Add weather data
mdf = pd.merge(bdf, wdf.drop(columns=['datetime', 'windir', 'details']), on=['date', 'hour'], how='left')
# Some hours may not have weather data related
# We impute them with the latest weather data available
mdf.sort_values(by=['date', 'hour'], ascending=[True, True], inplace=True)
mdf.fillna(method='ffill', inplace=True)

# Add public holiday data
mdf = mdf.merge(hdf, on=['date'], how='left')

# Save merged dataset
mdf.to_csv(os.path.join(os.pardir,'data', 'processed', 'merged_dataset.csv'), index=False)

### 0.5 Load merged dataset

In [6]:
mdf = pd.read_csv(os.path.join(os.pardir,'data', 'processed', 'merged_dataset.csv'))
# Re-convert date to datetime
mdf.date = pd.to_datetime(mdf.date)
mdf.head()

,date,hour,member,casual,total,temp,atemp,humidity,weather,windspeed,public_holiday
0,2010-09-20,11,2,0,2,24,23,31,2,19,0
1,2010-09-20,12,17,0,17,25,24,26,1,19,0
2,2010-09-20,13,11,0,11,26,25,24,1,24,0
3,2010-09-20,14,4,2,6,27,26,20,1,17,0
4,2010-09-20,15,10,2,12,27,26,20,1,26,0


### 1. Data pre-processing

### 1.1 Missing values

In [7]:
mdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63240 entries, 0 to 63239
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            63240 non-null  datetime64[ns]
 1   hour            63240 non-null  int64         
 2   member          63240 non-null  int64         
 3   casual          63240 non-null  int64         
 4   total           63240 non-null  int64         
 5   temp            63240 non-null  int64         
 6   atemp           63240 non-null  int64         
 7   humidity        63240 non-null  int64         
 8   weather         63240 non-null  int64         
 9   windspeed       63240 non-null  int64         
 10  public_holiday  63240 non-null  int64         
dtypes: datetime64[ns](1), int64(10)
memory usage: 5.3 MB


The dataset does not contain any missing values.

### 1.2 Outliers

In [8]:
mdf.describe()

,hour,member,casual,total,temp,atemp,humidity,weather,windspeed,public_holiday
count,63240.000000,63240.000000,63240.000000,63240.000000,63240.000000,63240.000000,63240.000000,63240.000000,63240.000000,63240.000000
mean,11.550870,236.276281,66.026739,302.303020,15.480266,14.814643,63.078495,2.874605,13.363694,0.035848
std,6.913238,255.359994,102.907403,319.765133,9.810974,11.617510,19.449951,0.984836,8.103467,0.185911
min,0.000000,0.000000,0.000000,1.000000,-14.000000,-24.000000,7.000000,1.000000,0.000000,0.000000
25%,6.000000,43.000000,5.000000,51.000000,8.000000,5.000000,48.000000,2.000000,7.000000,0.000000
50%,12.000000,161.000000,23.000000,198.000000,17.000000,17.000000,63.000000,3.000000,13.000000,0.000000
75%,18.000000,337.000000,85.000000,447.000000,23.000000,24.000000,79.000000,4.000000,19.000000,0.000000
max,23.000000,1639.000000,1037.000000,1988.000000,48.000000,50.000000,100.000000,4.000000,83.000000,1.000000


Descriptive statistics show no evident outliers.

### 1.3 Feature engineering

The attribute `date` can provide useful information that we could use as variables. We will create three new attributes based on date:
1. `is_weekend`: indicates whether the day is weekend or not.
2. `day_of_week`: numerical representation of the day of the week.
3. `season`: Numerical representation of the season based on specific date ranges. Please note that these date ranges are approximate and may not precisely align with astronomical or meteorological definitions of seasons. The seasons are defined as follows:
    - **Winter**: The period from December 1st to the last day of February.
    - **Spring**: The period from March 1st to the last day of May.
    - **Summer**: The period from June 1st to the last day of August.
    - **Autumn**: The period from September 1st to the last day of November.

In [9]:
# Create a column indicating whether the day is a weekday or weekend
mdf['is_weekend'] = mdf['date'].dt.dayofweek // 5  # 1 for weekend, 0 for weekday

# Create a column for the number of the day in the week (Monday: 0, Sunday: 6)
mdf['day_of_week'] = mdf['date'].dt.dayofweek

# Create a column for the season number
# {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Autumn'}
mdf['season'] = (mdf['date'].dt.month % 12 + 3) // 3
# mdf['season_name'] = mdf['season'].map({1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Autumn'})

### 1.4 Train-test split

For temporal data like hourly bike demand, it's important to maintain the temporal order while splitting the data into train and test sets. A common approach is to use a time-based split, where you allocate a certain percentage of the data to the training set and the remaining to the test set.

Given that your data spans from 2010 to 2017, here's a suggested train-test split you can use as a starting point:
- Reserve the earlier years, such as 2010 to 2016, for training (approx 72% of the data).
- Allocate the later years, such as 2016 and 2017, for testing (approx 28% of the data).
  
This split ensures that the training set contains data from the earlier years, allowing the model to learn patterns and trends. The test set, on the other hand, includes data from the later years, which helps evaluate the model's performance on unseen future data.

In [21]:
# Perform the train-test split
train_data = mdf[mdf.date.dt.year < 2016]
test_data = mdf[mdf.date.dt.year >= 2016]

# Save the train and test sets to CSV files
train_data.to_csv(os.path.join(os.pardir,'data', 'processed', 'train.csv'), index=False)
test_data.to_csv(os.path.join(os.pardir,'data', 'processed', 'test.csv'), index=False)

### 3. Exploratory data analysis (EDA)

### 4. Modelling

### 4.3 Temporal model